In [ ]:
import cellsim.POP3p2x2 as mm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from pandas import IndexSlice as idx
from multiprocessing import Pool

import c14
from c14.models import liver_new_mitosis as lm
with pd.HDFStore('point_est.pandas') as st:
    point_est = st['F_IS_spline_newploidy'] 
para=dict(point_est.loc['POP3p_2x2n_NPspline','median'].dropna())
m = lm.POP3p_2x2n_NP(dnatotal=lm.dnatotal_spline)
m.set_parameters_phy(para,mode='bayes')
ips =[]
T = np.arange(0,90,0.1)
for i in T:
    ips.append(m.calc_implicit_parameters(i))
ips = c14.tools.listofdict_to_dictofarray(ips)
t=T
def calc(i):


    t=T
    p = mm.parameters()
    p.d2n.value = para['d2n']
    p.d2x2n.value = para['d2x2n']
    p.d4n.value = para['d4n']
    p.g2x2nb2n.value = para['g2x2nb2n']
    p.k2nb2x2n.value = para['k2nb2x2n']
    p.k2nb4n.value = para['k2nb4n']

    #time dependet parameteres list of doubles no extrapolations, just linear interpolation
    p.g2nb2n_t.set(t,ips['g2nb2n'])
    p.k2x2nb4n_t.set(t, ips['k2x2nb4n'])
    p.k4nb4n_t.set(t, ips['k4nb4n'])
    p.k4nb2x2n_t.set(t, ips['k4nb2x2n'])
    p.k2x2nb2x2n_t.set(t, ips['k2x2nb2x2n'])

    #mm.test run sim(
    # parameters ... mm.parameters() instance the parameter values
    # delta_t    ... simulation time delta
    # detlt_t_out .. write delta time
    # max_t      ... max time of simulation
    # seed
    # n2         ... number of initial n2 cells
    # n4         ... number of initial n4 cells
    
    #Paper Version
    #res = mm.test(p,0.001,0.1,80,i,int(N0*ploidy(0) ),int(N0*(1-ploidy(0) )) )

    #Fast Version
    sim = mm.sim()  
    sim.run(p,0.001,dto,maxA,int(i),int(N0i*m.ploidy(0) ),int(N0i*m.ploidy2x2(0)), int(N0i*(1-m.ploidy2x2(0)-m.ploidy(0))) )
    #output sort of vector [0] n2 cells; [1] n4 cells;
    
    return np.array(sim.get_time_age())


In [ ]:

#Paper Version
#N0 = 1e6
#Fast
dto = 0.1
maxA = 80
N0 = 1e6
tt = np.arange(0,maxA,dto)

In [ ]:

NUMP = 25
N0i = N0/NUMP
'''
with Pool(14) as p:
    xs = p.map(calc, range(NUMP))
    
nall = np.sum(xs,axis=0)
n2s = nall[0]
n2x2s = nall[1]
n4s = nall[2]
'''

In [ ]:
#np.save('POP3p2x2.npy_newploidy', nall)

nall =np.load('POP3p2x2.npy_newploidy.npy')
n2s = nall[0]
n2x2s = nall[1]
n4s = nall[2]


In [ ]:
dnatotal_to_N = N0*(m.ploidy(0)+2*(1-m.ploidy(0)))/float(lm.dnatotal_spline(0)[0])

In [ ]:
plt.plot(tt,n2s.sum(axis=1)+2*n2x2s.sum(axis=1)+2*n4s.sum(axis=1))
plt.plot(tt,lm.dnatotal_spline(tt)[0]*dnatotal_to_N,'--')

In [ ]:
plt.plot(tt,n2s.sum(axis=1)+2*n2x2s.sum(axis=1)+2*n4s.sum(axis=1))
plt.plot(tt,lm.dnatotal_spline(tt)[0]*dnatotal_to_N,'--')

In [ ]:
plt.figure(0)
plt.plot(tt,n2s.sum(axis=1))
plt.figure(1)
plt.plot(tt,n2x2s.sum(axis=1))
plt.figure(2)
plt.plot(tt,n4s.sum(axis=1))
plt.figure(3)
plt.plot(tt,n2s.sum(axis=1)+2*n2x2s.sum(axis=1)+2*n4s.sum(axis=1))
plt.plot(tt,lm.dnatotal_spline(tt)[0]*dnatotal_to_N,'--')
plt.figure(4)
plt.plot(tt,m.ploidy(tt),'--')
plt.plot(tt,m.ploidy2x2(tt),'--')
Ntotal = n2s.sum(axis=1)+n2x2s.sum(axis=1)+n4s.sum(axis=1)
plt.plot(tt,n2s.sum(axis=1)/(Ntotal))
plt.plot(tt,n2x2s.sum(axis=1)/(Ntotal))
plt.plot(tt,n4s.sum(axis=1)/(Ntotal))


In [ ]:
rdf = pd.DataFrame(n2s)
rdf.columns.name = 'cell_age'
rdf.index.name = 'subject_age'
df = rdf.unstack()
df.name = 'cell_number'
df = df.reset_index()

df['cell_birth'] = df['subject_age'] - df['cell_age']

df = df.query("cell_birth >= 0")[['subject_age', 'cell_number', 'cell_birth']].pivot(index='cell_birth', columns='subject_age')

In [ ]:
rdf2x2 = pd.DataFrame(n2x2s)
rdf2x2.columns.name = 'cell_age'
rdf2x2.index.name = 'subject_age'
df2x2 = rdf2x2.unstack()
df2x2.name = 'cell_number'
df2x2 = df2x2.reset_index()

df2x2['cell_birth'] = df2x2['subject_age'] - df2x2['cell_age']

df2x2 = df2x2.query("cell_birth >= 0")[['subject_age', 'cell_number', 'cell_birth']].pivot(index='cell_birth', columns='subject_age')

In [ ]:
rdf4 = pd.DataFrame(n4s)
rdf4.columns.name = 'cell_age'
rdf4.index.name = 'subject_age'
df4 = rdf4.unstack()
df4.name = 'cell_number'
df4 = df4.reset_index()

df4['cell_birth'] = df4['subject_age'] - df4['cell_age']

df4 = df4.query("cell_birth >= 0")[['subject_age', 'cell_number', 'cell_birth']].pivot(index='cell_birth', columns='subject_age')

In [ ]:
i=500
np.sum(rdf.loc[i]/rdf.loc[i].sum()*rdf.columns/10)

In [ ]:
i=500
np.sum(rdf2x2.loc[i]/rdf2x2.loc[i].sum()*rdf2x2.columns/10)

In [ ]:
i=500
np.sum(rdf4.loc[i]/rdf4.loc[i].sum()*rdf4.columns/10)

In [ ]:
decades = np.arange(8, 0,-1)
colors = [(v, v, v) for v in np.linspace(0.1, 0.9, len(decades))]

In [ ]:
sns.set_context('talk')
sns

In [ ]:
plt.figure(figsize=(10,5))

TOTAL = df.sum()+df2x2.sum()+df4.sum()
for i, color in zip(decades, colors):
    l = plt.fill_between(df.columns.levels[1]/10, 
                         df.loc[0:i*100].sum()/TOTAL, color=color,lw=0)
    plt.fill_between(df.columns.levels[1]/10,
                     df.sum()/TOTAL,
                     (df.sum()+df2x2.loc[0:i*100].sum())/TOTAL,
                     color=l.get_facecolor(),
                     lw=0)
    plt.fill_between(df.columns.levels[1]/10,
                     (df.sum()+df2x2.sum())/TOTAL,
                     (df.sum()+df2x2.sum()+df4.loc[0:i*100].sum())/TOTAL,
                     color=l.get_facecolor(),
                     label=f'{i*10:.0f}',lw=0)
#     plt.plot(df4.columns.levels[1], df.sum()/(df.sum()+df4.sum())+df4.loc[0:i].sum()/(df.sum()+df4.sum()))
plt.legend(loc="upper left", bbox_to_anchor=(1.1, 1.1), title='birth decade')
plt.xlabel('subject age / years')
plt.ylabel('fraction')
plt.text(35, 0.4, '2n')
plt.text(35, 0.77, '2x2n')
plt.text(35, 0.9, '4n')
#plt.xlim(0, 80)
plt.ylim(0, 1)
plt.xlim(0,80)
plt.savefig('./age_distribution2x2_newploidy.pdf', bbox_inches='tight')
plt.show()

In [ ]:
plt.figure(figsize=(10,5))
TOTAL = (df.sum()+df2x2.sum()+df4.sum())
for i, color in zip(decades, colors):
    l = plt.fill_between(df.columns.levels[1]/10, 
                         df.loc[0:i*100].sum()/TOTAL, color=color,lw=0)
    plt.fill_between(df.columns.levels[1]/10,
                     (df.sum())/TOTAL,
                     (df.sum()+df2x2.loc[0:i*100].sum()+df4.loc[0:i*100].sum())/TOTAL,
                     color=l.get_facecolor(),
                     label=f'{i*10:.0f}',lw=0)
#     plt.plot(df4.columns.levels[1], df.sum()/(df.sum()+df4.sum())+df4.loc[0:i].sum()/(df.sum()+df4.sum()))
plt.legend(loc="upper left", bbox_to_anchor=(1.1, 1.1), title='birth decade')
plt.xlabel('subject age / years')
plt.ylabel('fraction')
plt.text(35, 0.4, '2n')
plt.text(35, 0.9, 'pn')
#plt.xlim(0, 80)
plt.ylim(0, 1)
plt.xlim(0,80)
plt.tight_layout()
plt.savefig('./age_distribution2x2_C_newploidy.pdf', bbox_inches='tight')
plt.show()

In [ ]:

x=(rdf.index/10)
total = (rdf+rdf4+rdf2x2)
y = ( (rdf.T/rdf.sum(axis=1)).T *rdf.columns/10).sum(axis=1)
y2= ( (rdf2x2.T/rdf2x2.sum(axis=1)).T * rdf2x2.columns/10).sum(axis=1)
y3= ( (rdf4.T/rdf4.sum(axis=1)).T *rdf4.columns/10).sum(axis=1)
y4 = ((total.T/total.sum(axis=1)).T *total.columns/10).sum(axis=1)
y5 = (((rdf4+rdf2x2).T/(rdf4+rdf2x2).sum(axis=1)).T *total.columns/10).sum(axis=1)
plt.plot(x,y,label='2n')
plt.plot(x,y2,label='2x2n')
plt.plot(x,y3,label='4n' )
plt.plot(x,y4,'--',label='all' )
plt.plot(x,y5,'--',label='pn' )
plt.legend()
plt.savefig('R2x2avgerage_age_newploidy.pdf')


In [ ]:
data = pd.DataFrame(np.array([y,y2,y3,y4,y5]).T,index=x,columns=['2n','2x2n','4n','all','pn'])
data.to_excel('R2x2average_cell_age_newploidy.xlsx')


In [ ]:
for i in [250,500,750]:
    y4 = rdf4.loc[i]/rdf4.loc[i].sum()
    print('Alter',i//10,' Wert',y4.loc[100:].sum())

In [ ]:
total = rdf2x2+rdf4
for i in [250,500,750]:
    ptotal = total.loc[i]/total.loc[i].sum()
    print('Alter',i//10,' Wert',ptotal.loc[100:].sum())

In [ ]:
total = rdf+rdf4+rdf2x2
for i in [20,250,500,750]:
    ptotal = total.loc[i]/total.loc[i].sum()
    print('Alter',i//10,' Wert',ptotal.loc[:20].sum())
    


In [ ]:
total = rdf
for i in [20,250,500,750]:
    ptotal = total.loc[i]/total.loc[i].sum()
    print('Alter',i//10,' Wert',ptotal.loc[:20].sum())
    


In [ ]:
import seaborn as sns

n2p = []
n2x2p = []
n4p = []
npn =[]
nall =[]
ages = np.array(rdf.columns)/10
for i in [250,500,750]:

    y = rdf.loc[i]/rdf.loc[i].sum()
    y2x2 = rdf2x2.loc[i]/rdf2x2.loc[i].sum()
    y4 = rdf4.loc[i]/rdf4.loc[i].sum()
    yall = rdf.loc[i]+rdf4.loc[i]+y2x2.loc[i]
    yall = yall/yall.sum()
    ypn = rdf4.loc[i]+rdf2x2.loc[i]
    ypn = ypn/ypn.sum()    
    
    tt =[]
    xx=np.arange(0,1,0.1)
    for p in xx:
        try:
            tt.append(ages[np.cumsum(y)<p][-1]+0.1)
        except IndexError:
            tt.append(0.0+0.1)
    n2p.append(pd.Series(tt,index=pd.Index(xx,name='percetile'),name=i/10))
  
    tt =[]
    xx=np.arange(0,1,0.1)
    for p in xx:
        try:
            tt.append(ages[np.cumsum(y2x2)<p][-1]+0.1)
        except IndexError:
            tt.append(0.0+0.1)
    n2x2p.append(pd.Series(tt,index=pd.Index(xx,name='percetile'),name=i/10))

    tt =[]
    xx=np.arange(0,1,0.1)
    for p in xx:
        try:
            tt.append(ages[np.cumsum(y4)<p][-1]+0.1)
        except IndexError:
            tt.append(0.0+0.1)
    n4p.append(pd.Series(tt,index=pd.Index(xx,name='percetile'),name=i/10))
    
    tt =[]
    xx=np.arange(0,1,0.1)
    for p in xx:
        try:
            tt.append(ages[np.cumsum(ypn)<p][-1]+0.1)
        except IndexError:
            tt.append(0.0+0.1)
    npn.append(pd.Series(tt,index=pd.Index(xx,name='percetile'),name=i/10))
    
    tt =[]
    xx=np.arange(0,1,0.1)
    for p in xx:
        try:
            tt.append(ages[np.cumsum(yall)<p][-1]+0.1)
        except IndexError:
            tt.append(0.0+0.1)
    nall.append(pd.Series(tt,index=pd.Index(xx,name='percetile'),name=i/10))  
    

n2p=pd.concat(n2p,axis=1)
n2x2p=pd.concat(n2x2p,axis=1)
n4p=pd.concat(n4p,axis=1)
npn=pd.concat(npn,axis=1)
nall=pd.concat(nall,axis=1)

tttidx=pd.Index(ages,name='person age')
n2ttt = pd.Series(np.diag(rdf.values/rdf.values.sum(axis=1)),index=tttidx,name='2n')
n2x2ttt = pd.Series(np.diag(rdf2x2.values/rdf2x2.values.sum(axis=1)),index=tttidx,name='2x2n')
n4ttt = pd.Series(np.diag(rdf4.values/rdf4.values.sum(axis=1)),index=tttidx,name='4n')

bb = rdf4 + rdf2x2
pnttt = pd.Series(np.diag(bb.values/bb.values.sum(axis=1)),index=tttidx,name='pn')
bb = rdf4 + rdf + rdf2x2
allttt = pd.Series(np.diag(bb.values/bb.values.sum(axis=1)),index=tttidx,name='all')
ages_b=pd.concat([n2ttt,n2x2ttt,n4ttt,pnttt,allttt],axis=1)

excel_writer = pd.ExcelWriter('agesR2x2_newploidy.xlsx')
n2p.to_excel(excel_writer=excel_writer,sheet_name='n2')
n2x2p.to_excel(excel_writer=excel_writer,sheet_name='n2x2')
n4p.to_excel(excel_writer=excel_writer,sheet_name='n4')
npn.to_excel(excel_writer=excel_writer,sheet_name='pn')
nall.to_excel(excel_writer=excel_writer,sheet_name='all')
ages_b.to_excel(excel_writer=excel_writer,sheet_name='ages')
excel_writer.save()



In [ ]:
f=plt.figure()
N=10
s=(N,1)
top = 3
ax =  plt.subplot2grid(s,(0,0),rowspan=top)
ax2 = plt.subplot2grid(s,(top,0),rowspan=N-top)

ax.set_ylim(10,65)
ax2.set_ylim(0,10)


#insgesammt auf eins normiert
xx = np.array([250,500,750])
d=0.2
d2=0
for i_n,i in enumerate(xx):
    cellhist = rdf.loc[i]/rdf.loc[i].sum()
    y = cellhist[cellhist!=0]
    x = ages[cellhist!=0]
    
    cellhist = (rdf2x2.loc[i]+rdf4.loc[i])/(rdf2x2.loc[i]+rdf4.loc[i]).sum()
    y4 = cellhist[cellhist!=0]
    x4 = ages[cellhist!=0]
    
    ax.fill_betweenx(x,  -y+i_n*d-d2  ,   i_n*d-d2,   color='gray',label='2n',lw=0.5,fc='gray')
    ax.fill_betweenx(x4,    i_n*d+d2 , y4+i_n*d+d2,     color='black',label='4n',lw=0.5,fc='black')
    ax2.fill_betweenx(x,  -y+i_n*d-d2  ,   i_n*d-d2,   color='gray',label='2n',lw=0.5,fc='gray')
    ax2.fill_betweenx(x4,    i_n*d+d2 , y4+i_n*d+d2,     color='black',label='4n',lw=0.5,fc='black')

ax2.set_xticks(np.arange(len(xx))*d)
ax2.set_xticklabels((xx/10).astype(int))
#ax2.set_xticklabels([''])
ax2.set_yticks([0,2,4,6,8])

ax.set_yticks([20,60])

# hide the spines between ax and ax2
ax.spines['bottom'].set_visible(False)
ax2.spines['top'].set_visible(False)
ax.tick_params(labelbottom='off')
ax.set_xticks([])

f.text(0.00, 0.5, 'cell age / years', va='center', rotation='vertical')
ax2.set_xlabel('subject age / years')


plt.subplots_adjust(hspace=0.1)
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[:2], labels[:2],loc="upper left")
d = .015 # how big to make the diagonal lines in axes coordinates
e = 0.02
kwargs = dict(transform=ax2.transAxes, color='k', clip_on=False)
ax2.plot((-d,+d), (1-d+e,1+d+e), **kwargs)
ax2.plot((1-d,1+d), (1-d+e,1+d+e), **kwargs)
ax2.plot((-d,+d), (1-d,1+d), **kwargs)
ax2.plot((1-d,1+d), (1-d,1+d), **kwargs)
plt.savefig('./R2x2_age_violinplot_normed_newploidy.pdf', bbox_inches='tight')


In [ ]:
f=plt.figure()
N=10
s=(N,1)
top = 3
ax =  plt.subplot2grid(s,(0,0),rowspan=top)
ax2 = plt.subplot2grid(s,(top,0),rowspan=N-top)

ax.set_ylim(10,65)
ax2.set_ylim(0,10)
#split axes n lower upper
xx = np.array([200])

#for Paper
#d=3e6
#d2=1e4

d2=0.00

#for i_n,i in enumerate(xx):
i=250
cell_hist=np.array([a for a in rdf.loc[i]+rdf2x2.loc[i]+rdf4.loc[i] ])/(rdf.loc[i]+rdf2x2.loc[i]+rdf4.loc[i]).sum()
y = cell_hist[cell_hist!=0]
x = ages[cell_hist!=0]
print(x[-1])
i=750
cell_hist=np.array([a for a in rdf.loc[i]+rdf2x2.loc[i]+rdf4.loc[i] ])/(rdf.loc[i]+rdf2x2.loc[i]+rdf4.loc[i]).sum()
y4 = cell_hist[cell_hist!=0]
x4 = ages[cell_hist!=0]
print(x4[-1])

ax2.fill_betweenx(x,  -y-d2  ,   -d2,   color='gray',label='25',lw=0.5,fc='gray')
ax2.fill_betweenx(x4,    d2 , y4+d2,     color='black',label='75',lw=0.5,fc='black')

ax.fill_betweenx(x,  -y-d2  ,   -d2,   color='gray',label='25',lw=0.5,fc='gray')
ax.fill_betweenx(x4,   +d2 , y4+d2,     color='black',label='75',lw=0.5,fc='black')




ax2.set_yticks([0,2,4,6,8])

ax.set_yticks([20,60])

# hide the spines between ax and ax2
ax.spines['bottom'].set_visible(False)
ax2.spines['top'].set_visible(False)
ax.tick_params(labelbottom='off')
ax.set_xticks([])

f.text(0.00, 0.5, 'cell age / years', va='center', rotation='vertical')
ax2.set_xlabel('')
ax2.set_xticks([])


plt.subplots_adjust(hspace=0.1)
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[:2], labels[:2],loc="upper left")
d = .015 # how big to make the diagonal lines in axes coordinates
e = 0.02
kwargs = dict(transform=ax2.transAxes, color='k', clip_on=False)
ax2.plot((-d,+d), (1-d+e,1+d+e), **kwargs)
ax2.plot((1-d,1+d), (1-d+e,1+d+e), **kwargs)
ax2.plot((-d,+d), (1-d,1+d), **kwargs)
ax2.plot((1-d,1+d), (1-d,1+d), **kwargs)


ax.set_xlim(ax.get_xlim()[0],-ax.get_xlim()[0])
ax2.set_xlim(ax2.get_xlim()[0],-ax2.get_xlim()[0])

plt.savefig('./R2x2_age_violinplot_ages_all_newploidy.pdf', bbox_inches='tight')